## ECS171 Final Project - Group 4

**Purpose**: Customer Analysis - to identify and separate cutomers into specific segments based on various attributes (age, income, etc) using unsupervised clustering methods
<br>**Dataset**: https://www.kaggle.com/imakash3011/customer-personality-analysis

#### Import needed libraries and load the dataset

In [ ]:
import sys
!{sys.executable} -m pip install altair delayed
!{sys.executable} -m pip install yellowbrick delayed
!{sys.executable} -m pip install mlxtend delayed

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from datetime import date, datetime
import altair as alt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# Loading the dataset
df = pd.read_csv("https://raw.githubusercontent.com/StephenW789/ECS-171-FinalProject/main/marketing_campaign.csv", sep = "\t")
print(df.head(5))

#### Data Cleaning and Feature Engineering
- perform exploratory data analysis to observe the given dataset by creating some data visualizations such as graphs and a heatmap
- remove rows with null values
- perform feature engineering by modifying some existing attributes and creating new attributes using the given attributes, e.g. Age of the customer, Total spendings on the different products, Total number of children, etc.
- drop redundant features and outliers

In [ ]:
# print data to see how the data needs to be processed
df.info()

In [ ]:
# DATA CLEANING
df_clean = df.copy()

# remove null values
df_clean = df_clean.dropna()

# Feature engineering
# reference: https://www.kaggle.com/karnikakapoor/customer-segmentation-clustering/notebook#DATA-CLEANING
# reference: https://thecleverprogrammer.com/2021/02/08/customer-personality-analysis-with-python/
# convert "Dt_Customer" to a datetime object
df['Dt_Customer'] = df['Dt_Customer'].apply(pd.to_datetime)
# Feature for number of days that customers started to shop
df['Customer_For'] = df['Dt_Customer'].apply(lambda x: (datetime.now() - x).days)
df_clean['Customer_For'] = df['Customer_For']
# print(df_clean['Customer_For'])

# Feature for age of customer today 
df_clean['Age'] = datetime.now().year - df['Year_Birth']
# print(df_clean['Age'])

# Feature for total spendings on various item categories
df_clean['Spent'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']
# print(df_clean['Spent'])

# Feature for simplifing marital status into 2 living situations: "Alone", "Partner"
df_clean['Living_With'] = df_clean['Marital_Status'].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})
# print(df_clean['Living_With'])

# Feature for total children living in the household
df_clean['Children'] = df_clean['Kidhome'] + df_clean['Teenhome']
# print(df_clean['Children'])

# Feature for total members in the household
df_clean['Family_Size'] = df_clean['Living_With'].replace({"Alone": 1, "Partner":2}) + df_clean['Children']
# print(df_clean['Family_Size'])

# Feature for whether the customer is a parent
df_clean['Is_Parent'] = np.where(df_clean.Children > 0, 1, 0)
# print(df_clean['Is_Parent'])

# Simplify education levels in three groups
df_clean['Education'] = df_clean['Education'].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})
# print(df_clean['Education'])

# Rename some columns for clarity
df_clean = df_clean.rename(columns = {"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Dropping some of the redundant features
to_drop = ['ID', 'Year_Birth', 'Marital_Status', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue']
df_clean = df_clean.drop(to_drop, axis=1)
print(df_clean.head(10))

In [ ]:
# DATA VISUALIZATION
# altair chart to check for any outliers
some_features = ['Income', 'Recency', 'Customer_For', 'Age', 'Spent']
alt.Chart(df_clean).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Is_Parent:N'
).properties(
    width=150,
    height=150
).repeat(
    row=some_features,
    column=some_features[::-1]
)

In [ ]:
# remove outliers from data
df_no_outliers = df_clean.copy()
df_no_outliers = df_no_outliers[(df_no_outliers['Income'] < 600000)]
# df_no_outliers = df_no_outliers[(df_no_outliers['Meat'] < 1500)]
# df_no_outliers = df_no_outliers[(df_no_outliers['Sweets'] < 250)]
df_no_outliers = df_no_outliers[(df_no_outliers['Age'] < 100)]

df_no_outliers.info()

In [ ]:
# ANOTHER DATA VISUALIZATION
# heatmap to see correlations between the different attributes
fig, ax = plt.subplots(figsize=(30,30))
print(sns.heatmap(df_no_outliers.corr(), annot=True, vmin=-1, vmax=1, center=0, linewidths=.5, cmap='coolwarm', ax=ax))

#### Data Preprocessing
- encode categorical attributes
- standardize all attributes 

In [ ]:
# DATA PREPROCESSING
df_encoded = df_no_outliers.copy()
# encode all categorical attributes
encoder = LabelEncoder()
df_encoded['Education'] = encoder.fit_transform(np.asarray(df_encoded['Education']))
# print(df_encoded['Education'].unique())
df_encoded['Living_With'] = encoder.fit_transform(np.asarray(df_encoded['Living_With']))
# print(df_encoded['Living_With'].unique())
# print(df_encoded.head(10))

# standardize all data attributes
scaler = StandardScaler()
df_scaled = df_encoded.copy()
cols_to_drop = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'Complain']
df_scaled = df_scaled.drop(columns = cols_to_drop)
df_scaled = pd.DataFrame(scaler.fit_transform(df_scaled), columns = df_scaled.columns)
print(df_scaled.head(10))

#### Dimensionality Reduction
- because we have a lot of attributes and the heatmap shows that some attributes have decent correlation with each other, we want to reduce the dimensionality by performing feature selection using PCA
- create Scree Plot to determine how many principal components should be included

In [ ]:
# apply a temporary pca on dataset with arbitrary number of components
# to see how many components should be included
pca_test = PCA(n_components = 5)
pca_test.fit(df_scaled)
df_pca_test = pd.DataFrame(pca_test.transform(df_scaled), columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

pca_values_test = np.arange(pca_test.n_components) + 1
plt.plot(pca_values_test, pca_test.explained_variance_, '-o', linewidth = 2)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

After looking at the Screen plot and doing some trial and error to determine the optimal number of principal components to include, we found that having 3 principal components gave the best distribution between clusters

In [ ]:
# dimensionality reduction
pca = PCA(n_components = 3)
pca.fit(df_scaled)
df_pca = pd.DataFrame(pca.transform(df_scaled), columns = ['PC1', 'PC2', 'PC3'])
PC1, PC2, PC3 = pca.components_
print(df_pca.head(10))
# see how the different attributes are weighed in the principal components
pd.set_option('display.max_columns', None)
display(pd.DataFrame(np.vstack((PC1.reshape(1,-1), PC2.reshape(1,-1), PC3.reshape(1,-1))), columns = df_scaled.columns, index = ['PC1', 'PC2', 'PC3']))

#### Method 1: Agglomerative Clustering (with results)

In [ ]:
# initiating the Agglomerative Clustering model 
# chose 4 clusters based on the optimal number of clusters for KMeans
AC = AgglomerativeClustering(n_clusters = 4)
# fit model and predict clusters
df_pca_ac = df_pca.copy()
yhat_AC = AC.fit_predict(df_pca_ac)
df_pca_ac["Clusters"] = yhat_AC

alt.Chart(df_pca_ac).mark_circle().encode(
    x="PC1",
    y="PC2",
    color="Clusters:N"
)

In [ ]:
# evaluation of the Agglomerative Clustering results
# looking at the distribution of the clusters
alt.Chart(df_pca_ac).mark_bar().encode(
    x="Clusters:N",
    y="count()",
)

In [ ]:
# adding the 'Clusters' feature to the original cleaned/encoded data
# so we can try to profile the different clusters
df_final_ac = df_encoded.copy()
df_final_ac["Clusters"]= yhat_AC


alt.Chart(df_final_ac).mark_circle().encode(
    x="Spent",
    y="Income",
    color="Clusters:N"
)

In [ ]:
alt.Chart(df_final_ac).mark_circle().encode(
    x="Age",
    y="Spent",
    color="Clusters:N"
)

In [ ]:
alt.Chart(df_final_ac).mark_circle().encode(
    x="Children",
    y="Spent",
    color="Clusters:N"
)

In [ ]:
# profile the different clusters (finding similarities in each group)
Personal = [ "Kidhome:O","Teenhome:O","Customer_For:Q", "Age:Q", "Children:O", "Family_Size:O", "Is_Parent:N", "Education:O","Living_With:O", "Income:Q"]

base = alt.Chart(df_final_ac).mark_bar().encode(
    y="count()",
    color="Clusters:N",
    facet="Clusters:N"
)

chart = alt.vconcat(data=df_final_ac)
for x_encoding in Personal:
    chart |= base.encode(x=alt.X(x_encoding, bin=True)).facet(row="Clusters:N")
chart

In [ ]:
alt.data_transformers.disable_max_rows()
# check to see how much of each type of product each group is consuming
columns = list(df_final_ac.columns)
values = ["Wines", "Fruits", "Meat", "Fish", "Sweets", "Gold"]
alt.Chart(df_final_ac.melt(id_vars=(set(columns) - set(values)),value_vars = values)).mark_bar(size=15).encode(
    y='median(value)',
    x='Clusters:N',
    color='Clusters:N',
    facet='variable:N'
)

#### Method 2: Apriori Algorithm (with results)

In [ ]:
# Create Age segment
cut_labels_Age = ['Young', 'Adult', 'Mature', 'Senior']
cut_bins = [0, 30, 45, 65, 120]
df_apriori = df_encoded.copy()
df_apriori['Age_group'] = pd.cut(df_apriori['Age'], bins=cut_bins, labels=cut_labels_Age)
#Create Income segment
cut_labels_Income = ['Low income', 'Low to medium income', 'Medium to high income', 'High income']
df_apriori['Income_group'] = pd.qcut(df_apriori['Income'], q=4, labels=cut_labels_Income)
#Create Seniority segment
cut_labels_Seniority = ['New customers', 'Discovering customers', 'Experienced customers', 'Old customers']
df_apriori['Seniority_group'] = pd.qcut(df_apriori['Customer_For'], q=4, labels=cut_labels_Seniority)
df_apriori = df_apriori.drop(columns=['Age','Income','Customer_For'])

cut_labels = ['Low consumer', 'Frequent consumer', 'Biggest consumer']
df_apriori['Wines_segment'] = pd.qcut(df_apriori['Wines'][df_apriori['Wines']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
df_apriori['Fruits_segment'] = pd.qcut(df_apriori['Fruits'][df_apriori['Fruits']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
df_apriori['Meat_segment'] = pd.qcut(df_apriori['Meat'][df_apriori['Meat']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
df_apriori['Fish_segment'] = pd.qcut(df_apriori['Fish'][df_apriori['Fish']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
df_apriori['Sweets_segment'] = pd.qcut(df_apriori['Sweets'][df_apriori['Sweets']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
df_apriori['Gold_segment'] = pd.qcut(df_apriori['Gold'][df_apriori['Gold']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
df_apriori.replace(np.nan, "Non consumer",inplace=True)
df_apriori.drop(columns=['Spent','Wines','Fruits','Meat','Fish','Sweets','Gold'],inplace=True)
df_apriori = df_apriori.astype(object)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 999)
pd.options.display.float_format = "{:.3f}".format
association = df_apriori[["Age_group", "Seniority_group", "Income_group", "Wines_segment", "Fruits_segment","Meat_segment", "Fish_segment", "Sweets_segment", "Gold_segment"]].copy() 
df_final_apriori = pd.get_dummies(association)
min_support = 0.08
max_len = 10
frequent_items = apriori(df_final_apriori, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

product='Wines'
segment='Biggest consumer'
target = '{\'%s_segment_%s\'}' %(product,segment)
results_personnal_care = rules[rules['consequents'].astype(str).str.contains(target, na=False)].sort_values(by='confidence', ascending=False)
results_personnal_care.head()

#### Method 3: K-Means Clustering (with results)

In [ ]:
# K-MEANS CLUSTERING

# determine the optimal number of clusters
test_kmeans = KMeans()
elbow_visualizer = KElbowVisualizer(test_kmeans, k = 10)
elbow_visualizer.fit(df_pca)
elbow_visualizer.show()

# build the K-Means clustering model
kmeans = KMeans(n_clusters=4, random_state=21)
my_kmeans_clusters = kmeans.fit_predict(df_pca)
centers = kmeans.cluster_centers_

df_pca_kmeans = df_pca.copy()
df_pca_kmeans['Clusters'] = my_kmeans_clusters

alt.Chart(df_pca_kmeans).mark_circle().encode(
    x="PC1",
    y="PC2",
    color="Clusters:N"
)

In [ ]:
# evaluation of the K-means clustering results
# by looking at the distribution of the clusters
alt.Chart(df_pca_kmeans).mark_bar().encode(
    x="Clusters:N",
    y="count()",
)

In [ ]:
# profiling the different clusters (finding similarities in each group)
# include the clusters in the original cleaned/encoded data so we can try to profile the different clusters
df_final_kmeans = df_encoded.copy()
df_final_kmeans['Clusters'] = my_kmeans_clusters

alt.Chart(df_final_kmeans).mark_circle().encode(
    x="Spent",
    y="Income",
    color="Clusters:N"
)

In [ ]:
alt.Chart(df_final_kmeans).mark_circle().encode(
    x="Age",
    y="Spent",
    color="Clusters:N"
)

In [ ]:
alt.Chart(df_final_kmeans).mark_circle().encode(
    x="Children",
    y="Spent",
    color="Clusters:N"
)

In [ ]:
Personal = [ "Kidhome:O","Teenhome:O","Customer_For:Q", "Age:Q", "Children:O", "Family_Size:O", "Is_Parent:N", "Education:O","Living_With:O", "Income:Q"]

base = alt.Chart(df_final_kmeans).mark_bar().encode(
    y="count()",
    color="Clusters:N",
    facet="Clusters:N"
)

chart = alt.vconcat(data=df_final_kmeans)
for x_encoding in Personal:
    chart |= base.encode(x=alt.X(x_encoding, bin=True)).facet(row="Clusters:N")
chart

In [ ]:
alt.data_transformers.disable_max_rows()

columns = list(df_final_kmeans.columns)
values = ["Wines", "Fruits", "Meat", "Fish", "Sweets", "Gold"]
alt.Chart(df_final_kmeans.melt(id_vars=(set(columns) - set(values)),value_vars = values)).mark_bar(size=15).encode(
    y='median(value)',
    x='Clusters:N',
    color='Clusters:N',
    facet='variable:N'
)

#### Method 4: Fuzzy C-Means Clustering (with results)

In [ ]:
np.random.seed(42)
# online definition to draw ellipse
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
# compute centroids
def compute_centroids():
    dummy = w.transpose()
    #x values
    for n in range(c):
        numerator = np.sum((dummy[n]**p)*col1)
        denominator = np.sum(dummy[n]**p)
        centroids[n,0] = numerator/denominator
    #y values
    for n in range(c):
        numerator = np.sum((dummy[n]**p)*col2)
        denominator = np.sum(dummy[n]**p)
        centroids[n,1] = numerator/denominator

In [ ]:
# update the fuzzy-pseudo partition
def fp_partition():
    dummy = np.zeros((c,size))
    for n in range(c):
        distance = (((col1-centroids[n,0])**2)+((col2-centroids[n,1])**2))**0.5
        dummy[n] = (distance**-1)**(1/(p-1))
    for m in range(c):
        for n in range(size):       
            numerator = dummy[m,n]
            denominator = np.sum(dummy.transpose()[n])
            w[n,m] = numerator/denominator

In [ ]:
# setup
size = df_encoded.shape[0]
col1 = df_pca["PC1"].to_numpy()
col2 = df_pca["PC2"].to_numpy()
c = 4 #number of clusters
p = 1.8
w = np.random.dirichlet(np.ones(c),size=size)
centroids = np.zeros((c,2))
compute_centroids()

In [ ]:
# repeat until w doesn't change
for n in range(200):
    fp_partition()
    compute_centroids()
#sum_squared_error()

In [ ]:
# assign each data point to the cluster with most membership
cluster = np.argmax(w, axis=1)
df_pca_fuzzyc = df_pca.copy()
new_data1 = df_pca_fuzzyc.assign(Clusters = cluster)
df_final_fuzzyc = df_encoded.copy()
new_data2 = df_final_fuzzyc.assign(Clusters = cluster)

In [ ]:
# plotting processed points
fig, ax = plt.subplots(1,1,figsize=(10,10))
color = ['tab:blue', 'tab:orange', 'tab:red', 'tab:cyan', 'tab:green', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:olive', 'tab:gray']
for n in range(c):
    dummy = new_data1.query('Clusters==%d'%(n))
    x = dummy["PC1"].to_numpy()
    y = dummy["PC2"].to_numpy()
    ax.scatter(x, y, c=color[n], s=5)
    confidence_ellipse(x, y, ax, edgecolor=color[n] ,ls='--' ,lw=1)
plt.show()

In [ ]:
# plotting countplot of clusters
pl = sns.countplot(x=new_data1["Clusters"])
plt.show()

In [ ]:
# plotting data points with color indicating the membership of each point
fig, ax = plt.subplots(1,1,figsize=(10,10))
color = ['tab:blue', 'tab:orange', 'tab:red', 'tab:cyan', 'tab:green', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:olive', 'tab:gray']
for n in range(c):
    dummy = new_data2.query('Clusters==%d'%(n))
    x = dummy["Spent"].to_numpy()
    y = dummy["Income"].to_numpy()
    ax.scatter(x, y, c=color[n], s=5)
    confidence_ellipse(x, y, ax, edgecolor=color[n] ,ls='--' ,lw=1)
plt.show()

In [ ]:
# graph different c value with increment=1
fig, axes = plt.subplots(3,3,figsize=(10,10))
axes = axes.reshape(-1)
size = df_encoded.shape[0]
col1 = df_pca["PC1"].to_numpy()
col2 = df_pca["PC2"].to_numpy()
color = ['tab:blue', 'tab:orange', 'tab:red', 'tab:cyan', 'tab:green', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:olive', 'tab:gray']
c = 2
for ax in axes:
    p = 1.5
    w = np.random.dirichlet(np.ones(c),size=size)
    centroids = np.zeros((c,2))
    compute_centroids()
    for n in range(200):
        fp_partition()
        compute_centroids()
    cluster = np.argmax(w, axis=1)
    new_data = df_pca_fuzzyc.assign(Clusters = cluster)
    for n in range(c):
        dummy = new_data.query('Clusters==%d'%(n))
        x = dummy["PC1"].to_numpy()
        y = dummy["PC2"].to_numpy()
        ax.scatter(x, y, c=color[n], s=1)
        confidence_ellipse(x, y, ax, edgecolor=color[n] ,ls='--' ,lw=1)
    ax.title.set_text("c = %d"%(c))
    c = c+1 #increment
fig.tight_layout()

In [ ]:
# graph different p value with increment=0.1
fig, axes = plt.subplots(4,4,figsize=(10,10),frameon=True)
axes = axes.reshape(-1)
size = df_encoded.shape[0]
col1 = df_pca["PC1"].to_numpy()
col2 = df_pca["PC2"].to_numpy()
p = 1.1
color = ['tab:blue', 'tab:orange', 'tab:red', 'tab:cyan', 'tab:green', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:olive', 'tab:gray']
for ax in axes:
    c = 4
    w = np.random.dirichlet(np.ones(c),size=size)
    centroids = np.zeros((c,2))
    compute_centroids()
    for n in range(200):
        fp_partition()
        compute_centroids()
    cluster = np.argmax(w, axis=1)
    new_data = df_pca_fuzzyc.assign(Clusters = cluster)
    for n in range(c):
        dummy = new_data.query('Clusters==%d'%(n))
        x = dummy["PC1"].to_numpy()
        y = dummy["PC2"].to_numpy()
        ax.scatter(x, y, c=color[n], s=1)
        confidence_ellipse(x, y, ax, edgecolor=color[n] ,ls='--' ,lw=1)
    ax.title.set_text("p = %.1f"%(p))
    p = p+0.1 #increment
fig.tight_layout()